In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# templete = PromptTemplate.from_template("{country_a}과 {country_b}의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?")
# prompt = templete.format(country_a="멕시코", country_b="태국")

# chat.predict(prompt)

# # import some message constructors
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

# messages = [
#   SystemMessage(content="You are a geography expert. And you only reply in Korean"),
#   AIMessage(content="Hi, my nick name is Liberty!"),
#   HumanMessage(content="남한과 북한의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?")
# ]

# chat.predict_messages(messages)

templete = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "Hi, my nick name is {name}!"),
        (
            "human",
            "{country_a}과 {country_b}의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?",
        ),
    ]
)

prompt = templete.format_messages(
    language="Korean", name="Socrates", country_a="Mexico", country_b="Thailand"
)

chat.predict_messages(prompt)

Mexico와 Thailand 사이의 거리는 대략 16,000km 정도입니다. 제 이름은 소크라테스입니다. 어떤 도움이 필요하신가요?

AIMessageChunk(content='Mexico와 Thailand 사이의 거리는 대략 16,000km 정도입니다. 제 이름은 소크라테스입니다. 어떤 도움이 필요하신가요?')

In [63]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))




template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}")
])

# prompt = template.format_messages(max_items=10, question="What are the colors?")

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question":"What are the poketmons?"})


pikachu, charmander, bulbasaur, squirtle, jigglypuff

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [64]:
investor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 세계적인 주식투자자야. 특히 한국 주식시장에서 스켈핑, 단타, 스윙에 아주 능숙해. 그리고 넌 뉴스를 읽고서 주가가 상승할 재료를 찾고 당일의 테마를 확인하는데 특출났어!",
        ),
        ("human", "과거와 오늘 뉴스들을 통해서 {stock_date} 주가 상승이 예상되는 종목과 테마를 찾고싶어.")
    ]
)

investor_chain = investor_prompt | chat




In [1]:
chart_investor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 세계적인 투자자이며 기술적 분석에 특화되어있어. 특정 종목이 급등할 때의 거래량과 차트가 어떤 현상이 나타나며 과거에 비슷한 history가 있었는지를 알고싶어. 그리고 평소와 어떻게 다른지도. 잘 모르겠다면 모르겠다고 해줘.",
        ),
        ("human", "{recipe}"),
    ]
)

chart_investor_chain = chart_investor_prompt | chat

final_chain = {"recipe": investor_chain} | chart_investor_chain
final_chain.invoke({"stock_date": "내일"})



NameError: name 'ChatPromptTemplate' is not defined

In [42]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    LengthBasedExampleSelector,
    FewShotPromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)


examples = [
    {
        "country": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples) -> None:
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables) -> List[dict]:
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")

example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables={"country"},
)

prompt.format(country="Brazil")


'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'